In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import LabelEncoder

In [2]:
# Set global parameters for the plots
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.labelsize'] = 12

Import all CSV files for the 2022/23 season

In [3]:
main_table_results = pd.read_csv('../../data/2022-23/main_table_results.csv')
match_table_results = pd.read_csv('../../data/2022-23/match_table_results.csv')
match_statistics = pd.read_csv('../../data/2022-23/match_statistics.csv')
players_statistics_home_team = pd.read_csv('../../data/2022-23/players_statistics_home_team.csv')
players_statistics_away_team = pd.read_csv('../../data/2022-23/players_statistics_away_team.csv')
match_officials = pd.read_csv('../../data/2022-23/match_officials.csv')

### EDA
Check in which tables we have missing values

In [4]:
main_table_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   position          20 non-null     int64 
 1   team              20 non-null     object
 2   played            20 non-null     int64 
 3   won               20 non-null     int64 
 4   drawn             20 non-null     int64 
 5   lost              20 non-null     int64 
 6   goals_for         20 non-null     int64 
 7   goals_against     20 non-null     int64 
 8   goals_difference  20 non-null     int64 
 9   points            20 non-null     int64 
dtypes: int64(9), object(1)
memory usage: 1.7+ KB


In [5]:
match_table_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   match_id    380 non-null    int64 
 1   home_team   380 non-null    object
 2   away_team   380 non-null    object
 3   score_ht    380 non-null    int64 
 4   score_at    380 non-null    int64 
 5   date_start  380 non-null    object
dtypes: int64(3), object(3)
memory usage: 17.9+ KB


In [6]:
match_statistics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   match_id          760 non-null    int64  
 1   stadium           760 non-null    object 
 2   attendance        760 non-null    int64  
 3   is_home_team      760 non-null    bool   
 4   team              760 non-null    object 
 5   possessions       760 non-null    float64
 6   total_shots       760 non-null    int64  
 7   on_target         760 non-null    int64  
 8   off_target        760 non-null    int64  
 9   blocked           760 non-null    int64  
 10  passing           760 non-null    float64
 11  clear_cut_chance  760 non-null    int64  
 12  corners           760 non-null    int64  
 13  offsides          760 non-null    int64  
 14  tackles           760 non-null    float64
 15  aerial_duels      760 non-null    float64
 16  saves             760 non-null    int64  
 1

In [7]:
players_statistics_home_team.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7595 entries, 0 to 7594
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   match_id              7595 non-null   int64  
 1   number_player         7595 non-null   int64  
 2   name_player           7595 non-null   object 
 3   nationality           7592 non-null   object 
 4   age                   7592 non-null   float64
 5   position              7592 non-null   object 
 6   transfer_fee          7462 non-null   float64
 7   first_team            7595 non-null   bool   
 8   substitution          7595 non-null   bool   
 9   play_time             7595 non-null   int64  
 10  goals                 7595 non-null   int64  
 11  penalty_goals         7595 non-null   int64  
 12  missed_penalty_goals  7595 non-null   int64  
 13  own_goals             7595 non-null   int64  
 14  assists               7595 non-null   int64  
 15  yellow_card          

In [8]:
players_statistics_away_team.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7590 entries, 0 to 7589
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   match_id              7590 non-null   int64  
 1   number_player         7590 non-null   int64  
 2   name_player           7590 non-null   object 
 3   nationality           7590 non-null   object 
 4   age                   7590 non-null   int64  
 5   position              7590 non-null   object 
 6   transfer_fee          7466 non-null   float64
 7   first_team            7590 non-null   bool   
 8   substitution          7590 non-null   bool   
 9   play_time             7590 non-null   int64  
 10  goals                 7590 non-null   int64  
 11  penalty_goals         7590 non-null   int64  
 12  missed_penalty_goals  7590 non-null   int64  
 13  own_goals             7590 non-null   int64  
 14  assists               7590 non-null   int64  
 15  yellow_card          

In [9]:
for table in [match_table_results, match_statistics,
              players_statistics_home_team, players_statistics_away_team, match_officials]:
    print(table['match_id'].nunique())

380
380
380
380
380


In [10]:
match_officials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3041 entries, 0 to 3040
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   match_id        3041 non-null   int64 
 1   name_officials  3040 non-null   object
 2   role            3041 non-null   object
dtypes: int64(1), object(2)
memory usage: 71.4+ KB


From the information above, we have that missing values are only present in the files:  
1. players_statistics_home_team
2. players_statistics_away_team 
3. match_officials

Now let's examine these rows with empty values

In [11]:
match_officials.loc[match_officials['name_officials'].isnull()]

,match_id,name_officials,role
2512,464714,NaN,Assistant VAR


Assistant VAR was not mentioned in the source, for some reasons, maybe it wasn't even present in that match, so let's remove this row from the table

In [12]:
match_officials.dropna(inplace=True)

In [13]:
match_officials.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3040 entries, 0 to 3040
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   match_id        3040 non-null   int64 
 1   name_officials  3040 non-null   object
 2   role            3040 non-null   object
dtypes: int64(1), object(2)
memory usage: 95.0+ KB


When parsing, there was an error with player number mismatches, for which additional data was supplemented with Null values. Therefore, if all four columns have missing values, we need to process them first since they have dependencies on previous values, and then handle the rest of the missing values

In [14]:
# We will label the rows to understand which row belongs to which table
players_statistics_home_team['source'] = 'home'
players_statistics_away_team['source'] = 'away'

home_team_with_date = players_statistics_home_team.merge(
    match_table_results, how='left', on='match_id')[players_statistics_home_team.columns.tolist() + ['date_start']]
away_team_with_date = players_statistics_away_team.merge(
    match_table_results, how='left', on='match_id')[players_statistics_away_team.columns.tolist() + ['date_start']]

all_players = pd.concat(
    [home_team_with_date, away_team_with_date], ignore_index=True
).sort_values(by=['name_player', 'date_start'], ascending=[True, True])

# We will remember the players names to correctly fill in the missing values based on previous and subsequent matches
all_players_is_null = all_players.loc[
    all_players[['nationality', 'age', 'position', 'transfer_fee']].isnull().all(axis=1)
]
filtered_players = all_players[all_players['name_player'].isin(all_players_is_null['name_player'])]

In [15]:
# Always fill the 'Nationality' column with the mode, grouped by players
filtered_players.loc[:, 'nationality'] = filtered_players['nationality'].fillna(
    filtered_players.groupby('name_player')['nationality'].transform(lambda x: x.mode().iloc[0])
)

# Visually, by opening the file 'filtered_players' and understanding how the data behaves
# (since it's not too extensive), we can figure out what to use for processing the missing values
grouped_players = filtered_players.groupby('name_player')
for column in ['transfer_fee', 'age', 'position']:
    filtered_players.loc[:, column] = grouped_players[column].fillna(method='ffill').fillna(method='bfill')

In [16]:
players_statistics_home_team = players_statistics_home_team.drop(columns='source')
players_statistics_away_team = players_statistics_away_team.drop(columns='source')

# We bring the processed df without missing values back to the format of the original table for data update
filtered_players_home = filtered_players[filtered_players['source'] == 'home'].drop(columns=['source', 'date_start'])
filtered_players_away = filtered_players[filtered_players['source'] == 'away'].drop(columns=['source', 'date_start'])

players_statistics_home_team.update(filtered_players_home)
players_statistics_away_team.update(filtered_players_away)

Now we need to perform similar actions only for the remaining column 'transfer_fee'

In [17]:
players_statistics_home_team['source'] = 'home'
players_statistics_away_team['source'] = 'away'

home_team_with_date = players_statistics_home_team.merge(
    match_table_results, how='left', on='match_id')[players_statistics_home_team.columns.tolist() + ['date_start']]
away_team_with_date = players_statistics_away_team.merge(
    match_table_results, how='left', on='match_id')[players_statistics_away_team.columns.tolist() + ['date_start']]

all_players = pd.concat(
    [home_team_with_date, away_team_with_date], ignore_index=True
).sort_values(by=['name_player', 'date_start'], ascending=[True, True])


all_players_is_null = all_players.loc[
    all_players[['transfer_fee']].isnull().any(axis=1)
]

filtered_players = all_players[all_players['name_player'].isin(all_players_is_null['name_player'])]

We will adhere to the following rule: If a player did not appear on the field throughout the entire season (game time 0), then in the further analysis, they will not provide any useful information. Therefore, we will simply discard such rows

In [18]:
grouped_players = filtered_players.groupby('name_player')


rows_to_drop = []
for name_player, group in grouped_players:
    column = ['first_team', 'substitution', 'play_time',
               'goals', 'penalty_goals', 'penalty_goals', 'own_goals',
               'assists', 'yellow_card', 'red_card']
    all_false_or_zero = all(group[column].sum() == 0)
    if all_false_or_zero:
        rows_to_drop.extend(group.index)


all_players = all_players.drop(rows_to_drop)
filtered_players = filtered_players.drop(rows_to_drop)

filtered_players_grouped = filtered_players.groupby('name_player')
filled_filtered_players = pd.DataFrame()

for name_player, group in filtered_players_grouped:
    if group['transfer_fee'].notnull().any():
        filled_group = group.ffill().bfill()
        filled_filtered_players = pd.concat([filled_filtered_players, filled_group])

all_players.update(filled_filtered_players)
        
all_players_home = all_players[all_players['source'] == 'home'].drop(columns=['source', 'date_start'])
all_players_away = all_players[all_players['source'] == 'away'].drop(columns=['source', 'date_start'])

filtered_players.update(filled_filtered_players)

median_transfer_fee = filtered_players['transfer_fee'].median()

all_players_home['transfer_fee'].fillna(median_transfer_fee, inplace=True)
all_players_away['transfer_fee'].fillna(median_transfer_fee, inplace=True)

filtered_players['transfer_fee'].fillna(median_transfer_fee, inplace=True)

players_statistics_home_team = all_players_home
players_statistics_away_team = all_players_away

convert_dict = {
    'match_id': 'int32',
    'number_player': 'int16',
    'age': 'int8',
    'first_team': 'bool',
    'substitution': 'bool',
    'play_time': 'int16',
    'goals': 'int8',
    'penalty_goals': 'int8',
    'missed_penalty_goals': 'int8',
    'own_goals': 'int8',
    'assists': 'int8',
    'yellow_card': 'bool',
    'red_card': 'bool'
}

players_statistics_home_team = players_statistics_home_team.astype(convert_dict)
players_statistics_away_team = players_statistics_away_team.astype(convert_dict)

In [19]:
players_statistics_home_team.to_csv('../../data/2022-23/players_statistics_home_team.csv', index=False)
players_statistics_away_team.to_csv('../../data/2022-23/players_statistics_away_team.csv', index=False)
match_officials.to_csv('../../data/2022-23/match_officials.csv', index=False)